In [1]:
import os
import time
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import alpha_vantage as av
from alpha_vantage.alphavantage import AlphaVantage
from alpha_vantage.techindicators import TechIndicators
from alpha_vantage.alphavantage import csv
from alpha_vantage.timeseries import TimeSeries as ts
import matplotlib.pyplot as plt

In [2]:
# Load .env enviroment variables
load_dotenv()

True

In [3]:
# Specify paper trading environment & set API keys
alpaca_paper_endpoint = os.getenv("ALPACA_PAPER_ENDPOINT")
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
# Create alpaca API object
alpaca_api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    alpaca_paper_endpoint,
    api_version = 'v2'
)
# Get Alpaca Account
account = alpaca_api.get_account()
# Get positions and check market open/closed
clock = alpaca_api.get_clock()
positions = alpaca_api.list_positions()
# Put positions and extra info into DataFrame
positions_df = pd.DataFrame({
    'ticker': [x.symbol for x in positions],
    'qty': [x.qty for x in positions],
    'lastday_price': [x.lastday_price for x in positions],
    'current_price': [x.current_price for x in positions],
    'change_today': [x.change_today for x in positions],
    'market_value': [x.market_value for x in positions],
    'unrealized_intraday_pl': [x.unrealized_intraday_pl for x in positions],
    'unrealized_intraday_plpc': [x.unrealized_intraday_plpc for x in positions],
})
positions_df

,ticker,qty,lastday_price,current_price,change_today,market_value,unrealized_intraday_pl,unrealized_intraday_plpc
0,RCL,100,67.63,67.6,-0.0004435901227266,6760,-3,-0.0004435901227266
1,NVDA,10,598.45,598.2,-0.0004177458434289,5982,-2.5,-0.0004177458434289
2,GOOG,10,2104.11,2104.11,0,21041.1,0,0
3,FVRR,100,323.1,325,0.0058805323429279,32500,190,0.0058805323429279
4,FTNT,50,164.73,164.73,0,8236.5,0,0
5,CPE,500,23.18,23.3,0.0051768766177739,11650,60,0.0051768766177739


In [4]:
positions_tickers=positions_df.ticker.tolist()
positions_tickers

['RCL', 'NVDA', 'GOOG', 'FVRR', 'FTNT', 'CPE']

In [5]:
# Get AlphaVantage API Key
av_api_key = os.getenv("ALPHAVANTAGE_API_KEY")
# Set AlphaVantage URL
av_api = 'https://www.alphavantage.co/query?'

In [6]:
# Define indicators amd indicator parameters for AlphaVantage API for each ticker
functions_list = ['SMA','EMA','RSI','MACD','BBANDS']
interval='daily'
time_period='10'
series_type='close'
response_dict = {}
function_responses={}

In [7]:
def get_position_indicators(functions_list, positions_tickers):
    print("Starting loops...\n")
    for x in positions_tickers:
        print(f'Starting {x} ticker')
        function_responses={}
        for fun in functions_list:
            print(f"Making query for {fun} for {x} ticker...")
            query={'function':fun,'symbol':x,'interval':interval,'time_period':time_period,'series_type':series_type,'apikey':av_api_key}
            print("Making the request...")
            response=requests.get(av_api,params=query)
            print("Converting to json...")
            response_dict = response.json()
            print("Saving to dict...")
            function_responses[fun]=response_dict
            print("Sleeping for 13 seconds...")
            time.sleep(13)
            print("Moving to next ticker... \n")
        print("FINAL... Saving to response dict...")
        response_dict[x]=function_responses
    return response_dict

In [8]:
# # Convert to DataFrame
# response_df = pd.DataFrame.from_dict(response_dict)
# response_df

In [11]:
function_responses

{}